In [1]:
from peaceful_pie.unity_comms import UnityComms
import argparse
import gym
from gym import Env
from gym.spaces import Box, MultiBinary,Discrete
import numpy as np
import os
from torch.utils.tensorboard import SummaryWriter
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
import gym
from gym import spaces
from gym.wrappers import FrameStack
from collections import deque
# Import UnityComms from peaceful_pie.unity_comms
from peaceful_pie.unity_comms import UnityComms
from peaceful_pie.unity_comms import UnityComms
from peaceful_pie import ray_results_helper
import pandas as pd
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torch.optim as optim

In [2]:
def unity_comms(port: int):
    unity_comms = UnityComms(port)
    return unity_comms


In [3]:
port = 5000  # Replace with your desired port number
unity_comms_instance = unity_comms(port)

In [4]:
unity_comms_instance 
unity_comms = unity_comms_instance

In [5]:
class MyVector3:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

In [6]:
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter



class UnityEnv(Env):
    def __init__(self, unity_comms, i):
        self.unity_comms = unity_comms
        self.action_space = Discrete(15)  # Scale the action range to -1.0 to 1.0
        self.observation_space = Box(low=-np.inf, high=np.inf, shape=(3,), dtype=np.float32)
        self.initial_position = None
        self.prev_position = None
        self.prev_velocity = 0
        self.initial_get_CarCollisionDetected = 0
        self.frame_count = 0
        self.obstacle_max = 0
        self.reward_max = 0
        self.i = i
        self.reward_modifier = self.ModifyReward()
        self.loop_counter = 0
        self.save_counter = 0
        self.modified_reward = 0
        self.global_step = 0
        self.summary_writer = SummaryWriter()
        self.reward_modifier.global_step = self.global_step
        self.reward_modifier.summary_writer = self.summary_writer

    class ModifyReward:
        def __init__(self):
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            self.model = nn.Sequential(
                nn.Linear(4, 32),
                nn.ReLU(),
                nn.Linear(32, 256),
                nn.ReLU(),
                nn.Linear(256, 1),
                nn.Tanh()
            ).to(self.device)
            #load model
            self.model.load_state_dict(torch.load('D:/RL_UNITY/full_car_envs/my_experiment/Assets/subAssets/Python/environment/Env5000/mini_model/mini_model_5000_6'))
            self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)
            self.dataset = []
            self.y_prde = None
            self.global_step = 0
            self.summary_writer = None

        def train(self):
            dataloader = DataLoader(self.dataset)
            for obs, action, rewards in dataloader:
                obs, action, rewards = obs.to(self.device), action.to(self.device), rewards.to(self.device)
                inputs = torch.cat((obs.squeeze(0), action), dim=1)
                self.y_prde = self.model(inputs)
                loss = nn.MSELoss()(self.y_prde, rewards.unsqueeze(1))
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                self.summary_writer.add_scalar("Loss", loss.item(), global_step=self.global_step)
                self.global_step += 1

            return self.y_prde.item()

        def store(self, obs, action, rewards):
            obs = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
            action = torch.tensor(action, dtype=torch.float32).unsqueeze(0)
            rewards = torch.tensor(rewards, dtype=torch.float32).unsqueeze(0)
            self.dataset.append((obs, action, rewards))

        def clear(self):
            self.dataset.clear()

    def step(self, action):
        # Perform the action based on the provided action index
        if action == 0:
            getattr(self.unity_comms, f"GoForward_{self.i}")()
        elif action == 1:
            getattr(self.unity_comms, f"GoReverse_{self.i}")()
        elif action == 2:
            getattr(self.unity_comms, f"TurnLeft_{self.i}")()
        elif action == 3:
            getattr(self.unity_comms, f"TurnRight_{self.i}")()
        elif action == 4:
            getattr(self.unity_comms, f"Handbrake_{self.i}")()
        elif action == 5:
            getattr(self.unity_comms, f"GoForward_{self.i}")()
            getattr(self.unity_comms, f"TurnLeft_{self.i}")()
        elif action == 6:
            getattr(self.unity_comms, f"GoForward_{self.i}")()
            getattr(self.unity_comms, f"TurnRight_{self.i}")()
        elif action == 7:
            getattr(self.unity_comms, f"GoForward_{self.i}")()
            getattr(self.unity_comms, f"Handbrake_{self.i}")()
        elif action == 8:
            getattr(self.unity_comms, f"GoReverse_{self.i}")()
            getattr(self.unity_comms, f"TurnLeft_{self.i}")()
        elif action == 9:
            getattr(self.unity_comms, f"GoReverse_{self.i}")()
            getattr(self.unity_comms, f"TurnRight_{self.i}")()
        elif action == 10:
            getattr(self.unity_comms, f"GoReverse_{self.i}")()
            getattr(self.unity_comms, f"Handbrake_{self.i}")()
        elif action == 11:
            getattr(self.unity_comms, f"GoForward_{self.i}")()
            getattr(self.unity_comms, f"TurnLeft_{self.i}")()
            getattr(self.unity_comms, f"Handbrake_{self.i}")()
        elif action == 12:
            getattr(self.unity_comms, f"GoForward_{self.i}")()
            getattr(self.unity_comms, f"TurnRight_{self.i}")()
            getattr(self.unity_comms, f"Handbrake_{self.i}")()
        elif action == 13:
            getattr(self.unity_comms, f"GoReverse_{self.i}")()
            getattr(self.unity_comms, f"TurnLeft_{self.i}")()
            getattr(self.unity_comms, f"Handbrake_{self.i}")()
        elif action == 14:
            getattr(self.unity_comms, f"GoReverse_{self.i}")()
            getattr(self.unity_comms, f"TurnRight_{self.i}")()
            getattr(self.unity_comms, f"Handbrake_{self.i}")()
        else:
            raise ValueError("Invalid action index")

        self.prev_position = self.Get_position()
        self.prev_velocity = self.Get_velocity()

        position = self.Get_position()
        velocity = self.Get_velocity()
        # Concatenate the position and velocity to form the observation
        self.observation = np.array([position[0], position[1], velocity], dtype=np.float32)

        reward = self.Get_reward()
        done = self.done()

        # Update the info dictionary with relevant information
        info = {
            'episode_reward': reward,
            'episode_length': self.frame_count,
            'current_observation': self.observation,  # Fix the variable name
            'action_taken': action,
            'obstacle_visible': self.obstacle_Collision_detected(),
            'goal_reached': self.Goal(),
            'car_collision_detected': self.Get_carCollisionDetected(),
            'Real_reward': self.pre_reward,
            'modified_reward': self.modified_reward,
            'loop_counter': self.loop_counter
        }

        self.reward_modifier.store(self.observation, action, reward)
        self.modified_reward=self.reward_modifier.train()  # Train the model before getting modified reward
        store = self.reward_modifier.store
        self.reward_modifier.clear()


        MINI_MODEL_DIR = './mini_model/'
        if not os.path.exists(MINI_MODEL_DIR):
            os.makedirs(MINI_MODEL_DIR)

            
        self.loop_counter += 1

        if self.loop_counter % 1000 == 0:
            self.save_counter += 1
            model_name = f"mini_model_{self.i}_{self.save_counter}"
            model_path = os.path.join(MINI_MODEL_DIR, model_name)
            torch.save(self.reward_modifier.model.state_dict(), model_path)
            print(f"Model saved as {model_name}")        
            # Write save_counter value to TensorBoard
            self.summary_writer.add_scalar("Save Counter", self.save_counter, global_step=self.global_step)

        return self.observation, reward, done, info
    
    
    def Get_reward(self):
        obstacle_penalty = -0.7  # Penalty for encountering an obstacle
        reward_bonus = 0.7  # Reward for finding a reward object
        goal_reward = 1  # Reward for reaching the goal
        collision_penalty = -1.0  # Penalty for collision
        car_collision_penalty = -1.0  # Penalty for colliding with the car
        velocity_reward = 0.1  # Reward for moving forward


        reward = 0.0
        if self.weight_comparision():
            reward += obstacle_penalty
        else:
            reward += reward_bonus

        reward += self.Step_panalty()

        if self.Goal():
            reward += goal_reward

        if self.obstacle_Collision_detected():
            reward += collision_penalty

        if self.Get_carCollisionDetected():
            reward += car_collision_penalty

        if self.Check_valocity_increment():
            reward += velocity_reward

        self.pre_reward = reward

        modified_reward = self.modified_reward
        return modified_reward
    


    def RayCast(self):
        ray_results = getattr(self.unity_comms, f"GetRayCastsResults_{self.i}")()
        distance = np.array(ray_results['rayDistances'], dtype=np.float32)
        types = np.array(ray_results['rayHitObjectTypes'], dtype=np.int32)
        num_types = ray_results['NumObjectTypes']
        self.actual_result = ray_results_helper.ray_results_to_feature_np(
            ray_results_helper.RayResults(
                NumObjectTypes=num_types,
                rayDistances=distance,
                rayHitObjectTypes=types,
            )
        )  

        return self.actual_result 
    

    def obstacle_visible(self):
        actual_result=self.RayCast()
        obstacle_channel = actual_result[0]  # Extract the obstacle channel from the actual result
        return obstacle_channel

    def reward_visible(self):
        actual_result=self.RayCast()
        reward_channel = actual_result[1]  # Extract the reward channel from the actual result
        return reward_channel
    
    def weight_comparision(self):

        obstacle_channel = self.obstacle_visible()
        reward_channel = self.reward_visible()
        self.obstacle_weight = np.mean(obstacle_channel)
        self.reward_weight = np.mean(reward_channel)

        if self.obstacle_weight > self.reward_weight:
            return True
        else:
            return False

    def Goal(self):
        reward_channel = self.reward_visible()
        reward_max = np.max(reward_channel)
        if reward_max >= 1:
            return True
        
        else:
            return False


    def obstacle_Collision_detected(self):
        obstacle_channel = self.obstacle_visible()
        obstacle_max = np.max(obstacle_channel)
        if obstacle_max >= 1:
            return True
        
        else:
            return False
        

    def reset(self):
        self.RayCast()
        Reset_Car = getattr(self.unity_comms, f"ResetPosition_{self.i}")()  
        Reset_Agent = getattr(self.unity_comms, f"ResetPosition_Plane_{self.i}")()
        position =self.Get_position()
        position = np.array(position, dtype=np.float32)
        self.frame_count=0
        self.reward_modifier.clear()
        return position

    def done(self):
        if self.Get_carCollisionDetected() == True:
            self.frame_count=0
            return True
        elif self.Get_movingPlaneCollision() == True:
            self.frame_count=0
            return True
        elif self.Get_planeCollision() == True:
            self.frame_count=0
            return True
        elif self.Check_stuck() == True:
            self.frame_count=0
            return True
        else:
            self.frame_count += 1
            return False

    def Get_carCollisionDetected(self):
        collision_count = 0
        collision = collision = getattr(self.unity_comms, f"CarCollisionDetected_{self.i}")()
        if collision == 1:
            collision_count += 1
            if collision_count >=2:
                collision_count = 0
                return True

        return False
    
    def Get_movingPlaneCollision(self):
        collision_count = 0
        collision = getattr(self.unity_comms, f"GetMovingPlaneCollision_{self.i}")()
        if collision == 1:
            collision_count += 1
            if collision_count >= 1:
                collision_count = 0
                return True

        return False
    
    def Get_planeCollision(self):
        collision_count = 0
        collision = getattr(self.unity_comms, f"GetPlaneCollision_{self.i}")()
        if collision == 1:
            collision_count += 1
            if collision_count > 0:
                collision_count = 0
                return True

        return False
    
    
    def Get_rewardCollision(self):
        collision = getattr(self.unity_comms, f"GetRewardCollision_{self.i}")()
        self.rewardcollision = collision

    def Step_panalty(self):
        if self.done() == True:
            return -1
        else:
            return 0


    def Get_velocity(self):
        valocity =  getattr(self.unity_comms, f"CarSpeedUI_{self.i}")()
        return valocity

    def Check_valocity_increment(self):
        valocity = self.Get_velocity()
        if valocity > self.prev_velocity:
            self.prev_velocity = valocity
            return True
        else:
            return False

    def Check_stuck(self):
        position_threshold = 0.01
        consecutive_steps = 40
        position_counter = 0
        
        if self.prev_position is None:
            self.prev_position = self.get_position()

        for _ in range(consecutive_steps):
            x, y, z = self.Get_position()

            position_diff = np.linalg.norm(np.array([x, y, z]) - np.array(self.prev_position))
            if position_diff < position_threshold:
                position_counter += 1
            else:
                position_counter = 0

            self.prev_position = [x, y, z]

        if position_counter >= consecutive_steps:
            return True
        else:
            return False


    def Get_position(self):
        position = getattr(self.unity_comms, f"GetPosition_{self.i}")()
        # Extract x, y, and z components from position
        x = position['x']
        y = position['y']
        z = position['z']
        return x, y, z




In [7]:
env = UnityEnv(unity_comms,5000)
env.reset()

array([2445.68    ,  -19.841257, -115.21    ], dtype=float32)

In [8]:
observation, reward, done, info = env.step(env.action_space.sample())

c:\Users\rifat\miniconda3\envs\tf\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1, 1, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [9]:
info

{'episode_reward': 1.7,
 'episode_length': 0,
 'current_observation': array([ 2.445680e+03, -2.067210e+01,  7.100961e-01], dtype=float32),
 'action_taken': 7,
 'obstacle_visible': False,
 'goal_reached': True,
 'car_collision_detected': False,
 'Real_reward': 1.7,
 'modified_reward': 0}

In [10]:
# Import optuna for HPO
import optuna
# Import PPO for algos
from stable_baselines3 import PPO
# Evaluate Policy
from stable_baselines3.common.evaluation import evaluate_policy
# Import wrappers
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
import os
LOG_DIR = './logs/'
if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)

OPT_DIR = './opt_modeldata/'
if not os.path.exists(OPT_DIR):
    os.makedirs(OPT_DIR)



c:\Users\rifat\miniconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Define the frame skip frequency
frame_skip_frequency = 5

# Define the number of training steps
total_timesteps = 100000

# Define the directory paths
LOG_DIR = './logs/'
OPT_DIR = './opt_modeldata/'
CHECKPOINT_DIR = './train_modeldata/'

# Create the directories if they don't exist
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(OPT_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)


In [12]:



# Import os for file path management
import os 
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback


class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True
    
CHECKPOINT_DIR = './train_modeldata/'
if not os.path.exists(CHECKPOINT_DIR):
    os.makedirs(CHECKPOINT_DIR)

callback = TrainAndLoggingCallback(check_freq=3000, save_path=CHECKPOINT_DIR)

env = UnityEnv(unity_comms,5000)
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')








In [16]:
from stable_baselines3 import PPO

model_params = {
    'n_steps': 500,
    'gamma': 0.839729258597705,
    'learning_rate': 4.456287316411457e-05,
    'clip_range': 0.29999077694725984,
    'gae_lambda': 0.9074598206209127
}


model = PPO('MlpPolicy', env,batch_size=4096, tensorboard_log=LOG_DIR, verbose=1, **model_params)

model.learn(total_timesteps=total_timesteps, callback=callback)

Using cuda device
Logging to ./logs/PPO_1


c:\Users\rifat\miniconda3\envs\tf\lib\site-packages\stable_baselines3\ppo\ppo.py:140: UserWarning: You have specified a mini-batch size of 4096, but because the `RolloutBuffer` is of size `n_steps * n_envs = 500`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 500
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=500 and n_envs=1)
  warnings.warn(
c:\Users\rifat\miniconda3\envs\tf\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1, 1, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.1      |
|    ep_rew_mean     | 2.99     |
| time/              |          |
|    fps             | 2        |
|    iterations      | 1        |
|    time_elapsed    | 245      |
|    total_timesteps | 500      |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 2.62          |
|    ep_rew_mean          | 6.19          |
| time/                   |               |
|    fps                  | 1             |
|    iterations           | 2             |
|    time_elapsed         | 530           |
|    total_timesteps      | 1000          |
| train/                  |               |
|    approx_kl            | 4.7984842e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.3           |
|    entropy_loss         | -2.71         |
|    explained_variance   | -0.365        |
